In [ ]:
import numpy as np
import os, sys

from skimage import io as skio
from skimage.morphology import disk as skidisk
from skimage.filters.rank import median as skimedian
from skimage import exposure as  skiexposure
from skimage.transform import downscale_local_mean
from skimage.filters.rank import enhance_contrast_percentile
from skimage.measure import label as skilabel
from skimage.measure import regionprops
from skimage.morphology import opening as skiopening

from sklearn.decomposition import PCA

from scipy import interpolate

from pyemd import emd as emdemd


def whaledet(image_location):
    
    # Various parameters
    n_emd_bins = 12
    emd_vs_median_ratio = 0.42
    emd_cutoff = 97.0
    contrast_disk_divider = 220.0
    gridsize = 0.018
    medfilt_disk_divider = 140.0
    mediandist_cutoff = 93.0
    open_disk_divider = 330.0
    whale_contrast_h = 0.94
    whale_contrast_l = 0.92
    whale_cutoff = 83.0
    
    # Amount to downscale the image
    downscale_first = 3
    downscale_second = 6
    
    # Read the image
    whaleimg = skio.imread(image_location)
    
    # Downsample image
    whaleimg = downscale_local_mean(whaleimg,(downscale_first,downscale_first,1))
    
    # Compute PCA
    whaleimgflat = whaleimg.reshape(-1, whaleimg.shape[2])
    pca = PCA(n_components=3)
    pca.fit(whaleimgflat)
    whaleimgpca = pca.transform(whaleimgflat)
    for i in range(3):
        whaleimgpca[:,i] = np.floor(255*(whaleimgpca[:,i]-whaleimgpca[:,i].min())/(whaleimgpca[:,i].max()-whaleimgpca[:,i].min())) 
    whaleimgpca = np.nan_to_num(whaleimgpca)
    whaleimgpca = whaleimgpca.astype(np.uint8)
    whaleimgpca[:,0] = 0 # Zero the component with the largest eigenvalue
    whaleimg = whaleimgpca.reshape(whaleimg.shape[0], whaleimg.shape[1], whaleimg.shape[2])
    whaleimgpca = None
    whaleimgflat = None
    
    # Compute grid sizes
    img_min_dim = np.minimum(whaleimg.shape[0],whaleimg.shape[1]).astype(np.float)
    n_v = np.floor(1.0/gridsize).astype(np.int)
    n_h = np.floor(1.0/gridsize).astype(np.int)
    
    if emd_vs_median_ratio < 1.0:
        # Get distance between median value of whole image and each pixel
        median_color = np.median(whaleimg.reshape(-1, whaleimg.shape[2]),axis=0)
        mediandist = np.linalg.norm(whaleimg-median_color,axis=2)
        mediandist = mediandist-np.percentile(mediandist,q=mediandist_cutoff,axis=(0,1))
        mediandist[mediandist<0] = 0
        mediandist = skiexposure.equalize_hist(mediandist)
    
    if emd_vs_median_ratio > 0.0:
        # Compute the distance between the mean emd components and the mean components from grid sections of the image
        imghist1, bins1 = np.histogram(whaleimg[:,:,1],bins=n_emd_bins,range=(0,255),density=True)
        imghist2, bins2 = np.histogram(whaleimg[:,:,2],bins=n_emd_bins,range=(0,255),density=True)
        histbins = bins1[:-1]
        darray = np.zeros((n_emd_bins,n_emd_bins),dtype=np.float)
        for i in range(n_emd_bins):
            for j in range(i,n_emd_bins):
                darray[i,j] = j-i
                darray[j,i] = j-i
        distances = np.zeros((n_v,n_h))
        v_segs = np.array_split(whaleimg,n_v,axis=0)
        for v in range(n_v):
            h_segs = np.array_split(v_segs[v],n_v,axis=1)
            for h in range(n_h):
                seghist1, bins1 = np.histogram(h_segs[h][:,:,1],bins=n_emd_bins,range=(0,255),density=True)
                seghist2, bins2 = np.histogram(h_segs[h][:,:,2],bins=n_emd_bins,range=(0,255),density=True)
                emddist1 = emdemd(imghist1,seghist1,darray)
                emddist2 = emdemd(imghist2,seghist2,darray)
                distances[v,h] = emddist1+emddist2
        v_segs = None
        h_segs = None
        imgseg = None
    
        distf = interpolate.interp2d(np.linspace(0,1,n_v),np.linspace(0,1,n_h),distances,kind='cubic')
        emddist = distf(np.linspace(0,1,whaleimg.shape[1]),np.linspace(0,1,whaleimg.shape[0]))
        distf = None
        emddist = emddist-np.percentile(emddist,q=emd_cutoff,axis=(0,1))
        emddist[emddist<0] = 0
        emddist = emddist/np.max(np.max(emddist))
    
    # Combine the two measures
    whaleimg = None
    if emd_vs_median_ratio == 1.0:
        detimg = emddist
    elif emd_vs_median_ratio == 0.0:
        detimg = mediandist
    else:
        detimg = ((1.0-emd_vs_median_ratio)*mediandist+emd_vs_median_ratio*emddist)
    mediandist = None
    emddist = None
    
    # Median filter
    detimg = np.floor(255* (detimg-detimg.max())/(detimg.max()-detimg.min()) ).astype(np.uint8)
    detimg = skimedian(detimg,skidisk(np.floor(img_min_dim/medfilt_disk_divider)))

    # Downsample again - remember to multiply this with the other one
    detimg = downscale_local_mean(detimg,(downscale_second,downscale_second)).astype(np.ubyte)
    img_min_dim /= float(downscale_second)

    # Threshold to find whale
    enh = enhance_contrast_percentile(detimg, skidisk(np.floor(img_min_dim/contrast_disk_divider)), p0=whale_contrast_l, p1=whale_contrast_h)
    enh = enh-np.percentile(enh,q=whale_cutoff,axis=(0,1))
    enh[enh<0] = 0
    enh[enh>0] = 1
    
    # Morphological open to remove bridges
    enhopen = np.zeros(enh.shape, dtype=np.float64)
    while enhopen.sum() <= 0:
        enhopen = skiopening(enh, skidisk(np.floor(img_min_dim/open_disk_divider)))
        open_disk_divider *= 1.25
        #print 'reopening: ' + str(open_disk_divider)

    # Choose largest object (region)
    (enhlab,nlabels) = skilabel(enhopen, connectivity=2, return_num=True)
    sizes = np.bincount(enhlab.ravel())
    sizes[sizes > enhlab.size/3] = 0 # Throw out objects bigger than a third of the image (background)
    enhlab[enhlab!=sizes.argmax()] = 0
    enhlab[enhlab>0] = 1
    enhopen = None

    # Get the region properties
    whale_region = regionprops(enhlab)
    enhlab = None
    
    # Try to return only one region
    if not whale_region:
        return ((np.nan,np.nan),(np.nan,np.nan))
    whale_region = whale_region[0] # Only one region
    
    # Get the longest axis of the region as an ellipse
    y0, x0 = whale_region.centroid
    orientation = whale_region.orientation
    x1 = x0 + np.cos(orientation) * 0.5 * whale_region.major_axis_length
    y1 = y0 - np.sin(orientation) * 0.5 * whale_region.major_axis_length
    x2 = x0 - np.cos(orientation) * 0.5 * whale_region.major_axis_length
    y2 = y0 + np.sin(orientation) * 0.5 * whale_region.major_axis_length
    x0 *= downscale_first * downscale_second
    y0 *= downscale_first * downscale_second
    x1 *= downscale_first * downscale_second
    y1 *= downscale_first * downscale_second
    x2 *= downscale_first * downscale_second
    y2 *= downscale_first * downscale_second

    return (int(x1),int(y1)),(int(x2),int(y2))

In [ ]:
import pandas as pd
import joblib

In [ ]:
df = pd.read_csv('./data/train.csv')
image_folder = './data/train'

df2 = pd.DataFrame(index=np.arange(len(df)), columns=['Image', 'x1', 'y1', 'x2', 'y2'])

def process(i):
    image_name = df.Image.ix[i]
    try:
        (x1,y1),(x2,y2) = whaledet(os.path.join(image_folder, image_name))
    except IndexError:
        x1 = y1 = x2 = y2 = np.nan
    return (i, (image_name, x1, y1, x2, y2))

results = joblib.Parallel(n_jobs=10)(joblib.delayed(process)(i) for i in range(len(df)))
for i, data in results:
    df2.loc[i] = data
    
df2.to_csv('./data/whaledet_train.csv')

In [ ]:
df = pd.read_csv('./data/sample_submission.csv')
image_folder = './data/test'

df2 = pd.DataFrame(index=np.arange(len(df)), columns=['Image', 'x1', 'y1', 'x2', 'y2'])

def process(i):
    image_name = df.Image.ix[i]
    try:
        (x1,y1),(x2,y2) = whaledet(os.path.join(image_folder, image_name))
    except IndexError:
        x1 = y1 = x2 = y2 = np.nan
    return (i, (image_name, x1, y1, x2, y2))

results = joblib.Parallel(n_jobs=10)(joblib.delayed(process)(i) for i in range(len(df)))
for i, data in results:
    df2.loc[i] = data
    
df2.to_csv('./data/whaledet_test.csv')